In [ ]:
!pip install --upgrade numpy tensorflow


In [2]:
import numpy as np
import tensorflow as tf
import pickle as pkl
from PIL import Image
from tensorflow.keras.applications.efficientnet import !!!i!!fytmnjh  jhjEfficientNetB0, preprocess_input
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

#### Constants
H, W = 224, 224
INPUT_SHAPE = (H, W, 3)
TARGET_SHAPE = (H, W)
NUM_CLASSES = 6

# Function to get class names
def get_class_names():
    with open('./class_list.pkl', 'rb') as f:
        class_names = pkl.load(f)
    return class_names

# Loading class names list
class_names = get_class_names()

# Function to get model ready
def get_model_ready():
    # Loading pre-trained EfficientNetB0
    efn = EfficientNetB0(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)

    # Making each layer not trainable
    for layer in efn.layers:
        layer.trainable = False

    # Input layer
    inputs = Input(shape=INPUT_SHAPE, name='input_shape')
    # Passing input layer to EfficientNetB0
    x = efn(inputs)
    # Global pooling layer
    x = GlobalAveragePooling2D(name='global_pooling')(x)
    # Dense layer
    x = Dense(1024, activation='relu', name='dense_1')(x)
    # Dropout layer
    x = Dropout(0.5, name='dropout_1')(x)
    # Dense layer
    x = Dense(64, activation='relu', name='dense_2')(x)
    # Dropout layer
    x = Dropout(0.5, name='dropout_2')(x)
    # Output layer
    outputs = Dense(NUM_CLASSES, activation='softmax', name='output')(x)

    # Initializing model
    efficientNet_model = Model(inputs=inputs, outputs=outputs, name='efficientNet_based_model')

    return efficientNet_model

# Loading trained model
model = get_model_ready()

# Loading trained weights
model.load_weights('./efficientNetB0_model.h5')

# Function to predict class labels on a single image
def final_fun_2(image):
    # Expanding dimensions to form (1, height, width, channel) format
    image = np.expand_dims(image, axis=0)    
    # Classifying image
    yhats = model.predict(image)
    # Returning class names index
    return yhats

# Function to preprocess and predict a single image
def predict_single_image(image_path):
    img = Image.open(image_path)
    img = img.resize(TARGET_SHAPE)
    img = np.array(img)
    img = preprocess_input(img)
    
    yhats = final_fun_2(img)
    pred = np.argmax(yhats, axis=1)[0]
    conf = np.round_(yhats[0][pred] * 100, 2)
    
    print(f"Image: {image_path}, Predicted: {class_names[pred]} with confidence: {conf}%")

# Example usage
if __name__ == "__main__":
    image_path = 'data/Bedroom/bed_1.jpg'  
    predict_single_image(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Image: data/Bedroom/bed_1.jpg, Predicted: bedroom with confidence: 99.77%


C:\Users\haric\AppData\Local\Temp\ipykernel_14224\1355006656.py:86: DeprecationWarning: `round_` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `round` instead.
  predict_single_image(image_path)
